## Trick: Use small samples to finish feature creating in order to increase speed

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import sklearn
import sqlite3
import datetime
from IPython.display import display 
import time
import pdb
%matplotlib inline

## 1 Load data and feature engineering

In [2]:
raw_data_store=pd.read_csv("data/store.csv")
raw_data_test=pd.read_csv("data/test.csv")
raw_data_train=pd.read_csv("data/train.csv",low_memory=False)

#deal with missing value using the median value
# The decison tree will handle the missing value automatically
# raw_data_store['CompetitionOpenSinceYear'].fillna(raw_data_store['CompetitionOpenSinceYear'].median(),inplace=True)
# raw_data_store['CompetitionOpenSinceYear']=raw_data_store['CompetitionOpenSinceYear'].astype(int)
# raw_data_store['CompetitionOpenSinceMonth'].fillna(raw_data_store['CompetitionOpenSinceMonth'].median(),inplace=True)
# raw_data_store['CompetitionOpenSinceMonth']=raw_data_store['CompetitionOpenSinceMonth'].astype(int)

raw_data_train=raw_data_train

raw_data_store=raw_data_store.set_index('Store',drop=False)
data_store=raw_data_store.copy()
data_train=raw_data_train.copy()

In [3]:
raw_data_store.head(n=5)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Store,,,,,,,,,,
1,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
3,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
4,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
5,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [4]:
raw_data_test.head(n=5)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [5]:
data_train.head(n=5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


#### feature only for data-store

In [6]:
#competition
data_store['LogCompetitionDistance']=np.log2( data_store['CompetitionDistance'])/(np.log2( data_store['CompetitionDistance'])).max()

#shop-week: mean,median, std,25%,75%
data_store['shopsales_mo_avg']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==1)]['Sales']).groupby(raw_data_train['Store']).mean())
data_store['shopsales_tu_avg']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==2)]['Sales']).groupby(raw_data_train['Store']).mean())
data_store['shopsales_we_avg']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==3)]['Sales']).groupby(raw_data_train['Store']).mean())
data_store['shopsales_th_avg']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==4)]['Sales']).groupby(raw_data_train['Store']).mean())
data_store['shopsales_fr_avg']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==5)]['Sales']).groupby(raw_data_train['Store']).mean())
data_store['shopsales_sa_avg']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==6)]['Sales']).groupby(raw_data_train['Store']).mean())

data_store['shopsales_mo_med']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==1)]['Sales']).groupby(raw_data_train['Store']).median())
data_store['shopsales_tu_med']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==2)]['Sales']).groupby(raw_data_train['Store']).median())
data_store['shopsales_we_med']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==3)]['Sales']).groupby(raw_data_train['Store']).median())
data_store['shopsales_th_med']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==4)]['Sales']).groupby(raw_data_train['Store']).median())
data_store['shopsales_fr_med']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==5)]['Sales']).groupby(raw_data_train['Store']).median())
data_store['shopsales_sa_med']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==6)]['Sales']).groupby(raw_data_train['Store']).median())

data_store['shopsales_mo_std']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==1)]['Sales']).groupby(raw_data_train['Store']).std())
data_store['shopsales_tu_std']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==2)]['Sales']).groupby(raw_data_train['Store']).std())
data_store['shopsales_we_std']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==3)]['Sales']).groupby(raw_data_train['Store']).std())
data_store['shopsales_th_std']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==4)]['Sales']).groupby(raw_data_train['Store']).std())
data_store['shopsales_fr_std']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==5)]['Sales']).groupby(raw_data_train['Store']).std())
data_store['shopsales_sa_std']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==6)]['Sales']).groupby(raw_data_train['Store']).std())

data_store['shopsales_mo_25%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==1)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.25))
data_store['shopsales_tu_25%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==2)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.25))
data_store['shopsales_we_25%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==3)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.25))
data_store['shopsales_th_25%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==4)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.25))
data_store['shopsales_fr_25%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==5)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.25))
data_store['shopsales_sa_25%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==6)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.25))

data_store['shopsales_mo_75%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==1)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.75))
data_store['shopsales_tu_75%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==2)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.75))
data_store['shopsales_we_75%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==3)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.75))
data_store['shopsales_th_75%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==4)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.75))
data_store['shopsales_fr_75%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==5)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.75))
data_store['shopsales_sa_75%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['DayOfWeek']==6)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.75))

#shop_customers_mean,median,std,25%,75%
data_store['shop_customers_mean']=pd.Series((raw_data_train[(raw_data_train['Open']==1)]['Customers']).groupby(raw_data_train['Store']).mean())
data_store['shop_customers_median']=pd.Series((raw_data_train[(raw_data_train['Open']==1)]['Customers']).groupby(raw_data_train['Store']).median())
data_store['shop_customers_std']=pd.Series((raw_data_train[(raw_data_train['Open']==1)]['Customers']).groupby(raw_data_train['Store']).std())
data_store['shop_customers_25%']=pd.Series((raw_data_train[(raw_data_train['Open']==1)]['Customers']).groupby(raw_data_train['Store']).quantile(q=0.25))
data_store['shop_customers_75%']=pd.Series((raw_data_train[(raw_data_train['Open']==1)]['Customers']).groupby(raw_data_train['Store']).quantile(q=0.75))

#shop_salespercustomer_mean,median,std,25%,75%
data_store['shop_salespercustomer']=1.0*(raw_data_train[(raw_data_train['Open']==1 )]['Sales'])/(raw_data_train[(raw_data_train['Open']==1 )]['Customers'])
data_store['shop_salespercustomer_mean']=pd.Series(data_store['shop_salespercustomer'].groupby(raw_data_train['Store']).mean())
data_store['shop_salespercustomer_median']=pd.Series(data_store['shop_salespercustomer'].groupby(raw_data_train['Store']).median())
data_store['shop_salespercustomer_std']=pd.Series(data_store['shop_salespercustomer'].groupby(raw_data_train['Store']).std())
data_store['shop_salespercustomer_25%']=pd.Series(data_store['shop_salespercustomer'].groupby(raw_data_train['Store']).quantile(q=0.25))
data_store['shop_salespercustomer_75%']=pd.Series(data_store['shop_salespercustomer'].groupby(raw_data_train['Store']).quantile(q=0.75))

#shop_schoolholiday_mean,median,std,25%,75%
data_store['shop_schoolholiday_mean']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['SchoolHoliday']==1)]['Sales']).groupby(raw_data_train['Store']).mean()) 
data_store['shop_schoolholiday_median']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['SchoolHoliday']==1)]['Sales']).groupby(raw_data_train['Store']).median()) 
data_store['shop_schoolholiday_std']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['SchoolHoliday']==1)]['Sales']).groupby(raw_data_train['Store']).std()) 
data_store['shop_schoolholiday_25%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['SchoolHoliday']==1)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.25))
data_store['shop_schoolholiday_75%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['SchoolHoliday']==1)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.75))

#shopsales_promo_mean,median,std,25%,75%
data_store['shopsales_promo_mean']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['Promo']==1)]['Sales']).groupby(raw_data_train['Store']).mean())
data_store['shopsales_promo_median']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['Promo']==1)]['Sales']).groupby(raw_data_train['Store']).median())
data_store['shopsales_promo_std']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['Promo']==1)]['Sales']).groupby(raw_data_train['Store']).std())
data_store['shopsales_promo_25%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['Promo']==1)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.25))
data_store['shopsales_promo_75%']=pd.Series((raw_data_train[(raw_data_train['Open']==1 )&(raw_data_train['Promo']==1)]['Sales']).groupby(raw_data_train['Store']).quantile(q=0.75))

/Users/pengjuzhao/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [7]:
data_store.head(n=5)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,LogCompetitionDistance,shopsales_mo_avg,shopsales_tu_avg,shopsales_we_avg,shopsales_th_avg,shopsales_fr_avg,shopsales_sa_avg,shopsales_mo_med,shopsales_tu_med,shopsales_we_med,shopsales_th_med,shopsales_fr_med,shopsales_sa_med,shopsales_mo_std,shopsales_tu_std,shopsales_we_std,shopsales_th_std,shopsales_fr_std,shopsales_sa_std,shopsales_mo_25%,shopsales_tu_25%,shopsales_we_25%,shopsales_th_25%,shopsales_fr_25%,shopsales_sa_25%,shopsales_mo_75%,shopsales_tu_75%,shopsales_we_75%,shopsales_th_75%,shopsales_fr_75%,shopsales_sa_75%,shop_customers_mean,shop_customers_median,shop_customers_std,shop_customers_25%,shop_customers_75%,shop_salespercustomer,shop_salespercustomer_mean,shop_salespercustomer_median,shop_salespercustomer_std,shop_salespercustomer_25%,shop_salespercustomer_75%,shop_schoolholiday_mean,shop_schoolholiday_median,shop_schoolholiday_std,shop_schoolholiday_25%,shop_schoolholiday_75%,shopsales_promo_mean,shopsales_promo_median,shopsales_promo_std,shopsales_promo_25%,shopsales_promo_75%
Store,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0.636024,5177.968750,4685.626866,4555.712121,4457.838710,4726.480620,4942.970149,5302.5,4640.5,4549.5,4380.0,4651.0,4785.0,1235.510826,1028.392921,922.143698,920.538973,912.926613,856.679364,4067.00,3817.00,3843.50,3799.50,4025.00,4353.50,5901.50,5401.50,5042.0,4964.50,5242.0,5314.5,564.049936,550.0,93.707476,500.00,612.00,9.702400,9.194139,9.194139,NaN,9.194139,9.194139,4796.325000,4562.5,1175.519034,4035.25,5474.25,5300.111429,5210.0,850.535714,4707.50,5694.00
2,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",0.564727,6061.953125,5369.201493,5834.946970,4967.472441,4669.589147,2873.104478,6530.5,5452.0,5920.5,5009.0,4671.0,2735.0,1831.746980,1506.142756,1174.602060,991.867935,825.742090,526.172651,4150.75,4077.75,4969.25,4111.00,4095.00,2553.50,7369.00,6366.25,6617.0,5568.50,5016.0,3010.5,583.998724,575.5,155.374483,497.75,687.25,10.126675,9.702400,9.702400,NaN,9.702400,9.702400,5324.849673,5127.0,1464.010246,4158.00,6315.00,6277.440678,6281.0,1182.105325,5373.75,6881.50
3,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",0.850437,8338.390625,7644.843284,7132.181818,6936.435484,7193.203125,4467.984962,8993.0,7928.0,7478.0,6940.0,7111.0,4221.0,2667.185783,2195.923041,1724.256071,1418.281002,1481.616010,1028.871631,5604.00,5599.25,5512.50,5697.25,6178.00,3845.00,10306.25,9059.75,8421.5,7909.25,8091.5,4725.0,750.077022,744.0,170.280290,660.50,863.00,9.342457,10.126675,10.126675,NaN,10.126675,10.126675,7386.743590,6829.5,2045.610911,5779.25,8744.50,8854.628571,8612.5,1474.803038,7862.00,9613.75
4,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,0.572210,10851.578125,9409.902985,8919.984848,9072.874016,9470.410853,10113.447761,11106.5,9599.5,8902.0,8899.0,9353.0,9918.5,2301.176656,1971.304644,1583.526129,1918.873107,1701.591020,1355.685857,8914.50,7653.50,7826.50,7812.50,8264.00,9331.25,12170.50,10565.25,9610.0,9830.00,10416.0,10566.5,1321.752551,1301.5,198.347844,1180.00,1429.25,8.626118,9.342457,9.342457,NaN,9.342457,9.342457,10124.823899,9839.0,2303.605869,8670.50,11102.00,10576.158640,10157.0,1732.956722,9340.00,11357.00
5,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,0.917173,6156.488189,5109.694030,5212.084615,4747.960317,4858.054688,2079.104478,6846.0,5193.5,5308.5,4774.0,4792.0,2009.5,2000.050499,1377.341279,1161.346202,1024.799980,848.609451,417.075952,4126.00,3846.50,4273.75,3870.50,4241.75,1805.50,7615.00,6119.25,5964.0,5244.00,5228.5,2204.5,537.340180,564.0,165.604598,484.50,645.00,9.594228,8.626118,8.626118,NaN,8.626118,8.626118,4958.538462,4708.5,1418.311672,3900.75,5674.25,6096.683761,5884.0,1210.949214,5188.00,6852.00


#### merge data_store and data_train

In [8]:
raw_data_train_store=pd.merge(data_train,data_store,on='Store')
data_train_store=raw_data_train_store.copy()
raw_data_train_store.head(n=5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,LogCompetitionDistance,shopsales_mo_avg,shopsales_tu_avg,shopsales_we_avg,shopsales_th_avg,shopsales_fr_avg,shopsales_sa_avg,shopsales_mo_med,shopsales_tu_med,shopsales_we_med,shopsales_th_med,shopsales_fr_med,shopsales_sa_med,shopsales_mo_std,shopsales_tu_std,shopsales_we_std,shopsales_th_std,shopsales_fr_std,shopsales_sa_std,shopsales_mo_25%,shopsales_tu_25%,shopsales_we_25%,shopsales_th_25%,shopsales_fr_25%,shopsales_sa_25%,shopsales_mo_75%,shopsales_tu_75%,shopsales_we_75%,shopsales_th_75%,shopsales_fr_75%,shopsales_sa_75%,shop_customers_mean,shop_customers_median,shop_customers_std,shop_customers_25%,shop_customers_75%,shop_salespercustomer,shop_salespercustomer_mean,shop_salespercustomer_median,shop_salespercustomer_std,shop_salespercustomer_25%,shop_salespercustomer_75%,shop_schoolholiday_mean,shop_schoolholiday_median,shop_schoolholiday_std,shop_schoolholiday_25%,shop_schoolholiday_75%,shopsales_promo_mean,shopsales_promo_median,shopsales_promo_std,shopsales_promo_25%,shopsales_promo_75%
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0.636024,5177.96875,4685.626866,4555.712121,4457.83871,4726.48062,4942.970149,5302.5,4640.5,4549.5,4380.0,4651.0,4785.0,1235.510826,1028.392921,922.143698,920.538973,912.926613,856.679364,4067.0,3817.0,3843.5,3799.5,4025.0,4353.5,5901.5,5401.5,5042.0,4964.5,5242.0,5314.5,564.049936,550.0,93.707476,500.0,612.0,9.7024,9.194139,9.194139,NaN,9.194139,9.194139,4796.325,4562.5,1175.519034,4035.25,5474.25,5300.111429,5210.0,850.535714,4707.5,5694.0
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0.636024,5177.96875,4685.626866,4555.712121,4457.83871,4726.48062,4942.970149,5302.5,4640.5,4549.5,4380.0,4651.0,4785.0,1235.510826,1028.392921,922.143698,920.538973,912.926613,856.679364,4067.0,3817.0,3843.5,3799.5,4025.0,4353.5,5901.5,5401.5,5042.0,4964.5,5242.0,5314.5,564.049936,550.0,93.707476,500.0,612.0,9.7024,9.194139,9.194139,NaN,9.194139,9.194139,4796.325,4562.5,1175.519034,4035.25,5474.25,5300.111429,5210.0,850.535714,4707.5,5694.0
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0.636024,5177.96875,4685.626866,4555.712121,4457.83871,4726.48062,4942.970149,5302.5,4640.5,4549.5,4380.0,4651.0,4785.0,1235.510826,1028.392921,922.143698,920.538973,912.926613,856.679364,4067.0,3817.0,3843.5,3799.5,4025.0,4353.5,5901.5,5401.5,5042.0,4964.5,5242.0,5314.5,564.049936,550.0,93.707476,500.0,612.0,9.7024,9.194139,9.194139,NaN,9.194139,9.194139,4796.325,4562.5,1175.519034,4035.25,5474.25,5300.111429,5210.0,850.535714,4707.5,5694.0
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0.636024,5177.96875,4685.626866,4555.712121,4457.83871,4726.48062,4942.970149,5302.5,4640.5,4549.5,4380.0,4651.0,4785.0,1235.510826,1028.392921,922.143698,920.538973,912.926613,856.679364,4067.0,3817.0,3843.5,3799.5,4025.0,4353.5,5901.5,5401.5,5042.0,4964.5,5242.0,5314.5,564.049936,550.0,93.707476,500.0,612.0,9.7024,9.194139,9.194139,NaN,9.194139,9.194139,4796.325,4562.5,1175.519034,4035.25,5474.25,5300.111429,5210.0,850.535714,4707.5,5694.0
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0.636024,5177.96875,4685.626866,4555.712121,4457.83871,4726.48062,4942.970149,5302.5,4640.5,4549.5,4380.0,4651.0,4785.0,1235.510826,1028.392921,922.143698,920.538973,912.926613,856.679364,4067.0,3817.0,3843.5,3799.5,4025.0,4353.5,5901.5,5401.5,5042.0,4964.5,5242.0,5314.5,564.049936,550.0,93.707476,500.0,612.0,9.7024,9.194139,9.194139,NaN,9.194139,9.194139,4796.325,4562.5,1175.519034,4035.25,5474.25,5300.111429,5210.0,850.535714,4707.5,5694.0


#### data_train_store: extract time

In [9]:
# # calc ThisDay,ThisMonth,ThisYear,Today_ISOWeek,Today_ISOYear,DaysOfYear
# start_time = time.time()

# data_train_store['ThisDay']=raw_data_train_store['Date'].apply(lambda x: (datetime.datetime.strptime(str(x), "%Y-%m-%d")).day)
# data_train_store['ThisMonth']=raw_data_train_store['Date'].apply(lambda x: (datetime.datetime.strptime(str(x), "%Y-%m-%d")).month)
# data_train_store['ThisYear']=raw_data_train_store['Date'].apply(lambda x: (datetime.datetime.strptime(str(x), "%Y-%m-%d")).year)
# def calc_ISOWeek(year,month,day):
#     return datetime.date(year,month,day).isocalendar()[1]
# def calc_ISOYear(year,month,day):
#     return datetime.date(year,month,day).isocalendar()[0]

# def calc_DaysOfYear(year,month,day):
#     return (datetime.date(year,month,day)-datetime.date(year,1,1)).days


# data_train_store['Today_ISOWeek']=data_train_store.apply(
#     lambda  row: calc_ISOWeek(row['ThisYear'],row['ThisMonth'],row['ThisDay']),axis=1)
# data_train_store['Today_ISOYear']=data_train_store.apply(
#     lambda  row: calc_ISOYear(row['ThisYear'],row['ThisMonth'],row['ThisDay']),axis=1)


# data_train_store['DaysOfYear']=data_train_store.apply(
#     lambda  row: calc_DaysOfYear(row['ThisYear'],row['ThisMonth'],row['ThisDay']),axis=1)
# print("--- %s seconds ---" % (time.time() - start_time))

# # calc Day_LastWeek,Month_LastWeek,Year_LastWeek,LastWeek_ISOWeek,LastWeek_ISOYear
# start_time = time.time()
# def calc_Day_LastWeek(year,month,day):
#     ThisDate=datetime.date(year,month,day)
#     Date_LastWeek=ThisDate-datetime.timedelta(7)
#     return Date_LastWeek.day

# def calc_Month_LastWeek(year,month,day):
#     ThisDate=datetime.date(year,month,day)
#     Date_LastWeek=ThisDate-datetime.timedelta(7)
#     return Date_LastWeek.month

# def calc_Year_LastWeek(year,month,day):
#     ThisDate=datetime.date(year,month,day)
#     Date_LastWeek=ThisDate-datetime.timedelta(7)
#     return Date_LastWeek.year


# data_train_store['Day_LastWeek']=data_train_store.apply(
#     lambda  row: calc_Day_LastWeek(row['ThisYear'],row['ThisMonth'],row['ThisDay']),axis=1)

# data_train_store['Month_LastWeek']=data_train_store.apply(
#     lambda  row: calc_Month_LastWeek(row['ThisYear'],row['ThisMonth'],row['ThisDay']),axis=1)

# data_train_store['Year_LastWeek']=data_train_store.apply(
#     lambda  row: calc_Year_LastWeek(row['ThisYear'],row['ThisMonth'],row['ThisDay']),axis=1)

# data_train_store['LastWeek_ISOWeek']=data_train_store.apply(
#     lambda  row: calc_ISOWeek(row['Year_LastWeek'],row['Month_LastWeek'],row['Day_LastWeek']),axis=1)

# data_train_store['LastWeek_ISOYear']=data_train_store.apply(
#     lambda  row: calc_ISOYear(row['Year_LastWeek'],row['Month_LastWeek'],row['Day_LastWeek']),axis=1)


# print("--- %s seconds ---" % (time.time() - start_time))
# # calc Day_NextWeek,Month_NextWeek,Year_NextWeek,NextWeek_ISOWeek,NextWeek_ISOYear
# start_time = time.time()
# def calc_Day_NextWeek(year,month,day):
#     ThisDate=datetime.date(year,month,day)
#     Date_NextWeek=ThisDate+datetime.timedelta(7)
#     return Date_NextWeek.day

# def calc_Month_NextWeek(year,month,day):
#     ThisDate=datetime.date(year,month,day)
#     Date_NextWeek=ThisDate+datetime.timedelta(7)
#     return Date_NextWeek.month

# def calc_Year_NextWeek(year,month,day):
#     ThisDate=datetime.date(year,month,day)
#     Date_NextWeek=ThisDate+datetime.timedelta(7)
#     return Date_NextWeek.year

# data_train_store['Day_NextWeek']=data_train_store.apply(
#     lambda  row: calc_Day_NextWeek(row['ThisYear'],row['ThisMonth'],row['ThisDay']),axis=1)

# data_train_store['Month_NextWeek']=data_train_store.apply(
#     lambda  row: calc_Month_NextWeek(row['ThisYear'],row['ThisMonth'],row['ThisDay']),axis=1)

# data_train_store['Year_NextWeek']=data_train_store.apply(
#     lambda  row: calc_Year_NextWeek(row['ThisYear'],row['ThisMonth'],row['ThisDay']),axis=1)

# data_train_store['NextWeek_ISOWeek']=data_train_store.apply(
#     lambda  row: calc_ISOWeek(row['Year_NextWeek'],row['Month_NextWeek'],row['Day_NextWeek']),axis=1)

# data_train_store['NextWeek_ISOYear']=data_train_store.apply(
#     lambda  row: calc_ISOYear(row['Year_NextWeek'],row['Month_NextWeek'],row['Day_NextWeek']),axis=1)

# print("--- %s seconds ---" % (time.time() - start_time))

In [10]:
#Calc ThisDay,ThisMonth,ThisYear,ThisWeek,DaysOfYear from 'Date'
start_time = time.time()

def calc_Day_Month_Year_ISOYear_ISOWeek_This_Next_Last_DaysOfYears(input_date):

    temp_date=datetime.datetime.strptime(str(input_date), "%Y-%m-%d")
    
    ThisDay=temp_date.day
    ThisMonth=temp_date.month
    ThisYear=temp_date.year
    ThisDate=datetime.date(ThisYear,ThisMonth,ThisDay)
    Today_ISOWeek=ThisDate.isocalendar()[1]
    Today_ISOYear=ThisDate.isocalendar()[0]
    DaysOfYear=(ThisDate-datetime.date(ThisYear,1,1)).days

    Date_LastWeek=ThisDate-datetime.timedelta(7)
    Date_NextWeek=ThisDate+datetime.timedelta(7)
    
    Day_LastWeek=Date_LastWeek.day
    Month_LastWeek=Date_LastWeek.month
    Year_LastWeek=Date_LastWeek.year
    LastWeek_ISOWeek=Date_LastWeek.isocalendar()[1]
    LastWeek_ISOYear=Date_LastWeek.isocalendar()[0]    
    
    Day_NextWeek=Date_NextWeek.day
    Month_NextWeek=Date_NextWeek.month
    Year_NextWeek=Date_NextWeek.year
    NextWeek_ISOWeek=Date_NextWeek.isocalendar()[1]
    NextWeek_ISOYear=Date_NextWeek.isocalendar()[0]    

        
    return pd.Series([ThisDay,ThisMonth,ThisYear,Today_ISOWeek,Today_ISOYear,DaysOfYear,Day_LastWeek,Month_LastWeek,Year_LastWeek,LastWeek_ISOWeek,LastWeek_ISOYear,Day_NextWeek,Month_NextWeek,Year_NextWeek,NextWeek_ISOWeek,NextWeek_ISOYear],
                     index=['ThisDay','ThisMonth','ThisYear','Today_ISOWeek','Today_ISOYear','DaysOfYear','Day_LastWeek','Month_LastWeek','Year_LastWeek','LastWeek_ISOWeek','LastWeek_ISOYear','Day_NextWeek','Month_NextWeek','Year_NextWeek','NextWeek_ISOWeek','NextWeek_ISOYear']
                    )
    
# temp_Day_Month_Year_Week_This_Next_Last_DaysOfYears=
temp_Day_Month_Year_ISOYear_ISOWeek_This_Next_Last_DaysOfYears=(data_train_store['Date']).apply(calc_Day_Month_Year_ISOYear_ISOWeek_This_Next_Last_DaysOfYears) 
data_train_store=data_train_store.merge(temp_Day_Month_Year_ISOYear_ISOWeek_This_Next_Last_DaysOfYears,left_index=True,right_index=True)
print("--- %s seconds ---" % (time.time() - start_time))


--- 326.097003937 seconds ---


In [11]:
start_time = time.time()
def calc_OpenDaysSinceCompetition(thisYear,thisMonth,thisDay,CompetitionOpenSinceYear,CompetitionOpenSinceMonth):
    return (datetime.date(thisYear,thisMonth,thisDay)-datetime.date(CompetitionOpenSinceYear,CompetitionOpenSinceMonth,1)).days


data_train_store['OpenDaysSinceCompetition']=data_train_store.apply(
    lambda  row: calc_OpenDaysSinceCompetition(row['ThisYear'],row['ThisMonth'],row['ThisDay'],row['CompetitionOpenSinceYear'],row['CompetitionOpenSinceMonth']),axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

TypeError: ('integer argument expected, got float', u'occurred at index 0')

In [ ]:
start_time = time.time()
def calc_DaysSincePromo2(thisYear,thisMonth,thisDay,Promo2,Promo2SinceYear,Promo2SinceWeek):
    if not Promo2:
        return 0
    else:
        Promo2SinceDate=datetime.datetime.strptime(str(int(Promo2SinceYear))+"-W"+str(int(Promo2SinceWeek))+"-1", "%Y-W%W-%w").date()
        TodayDate=datetime.date(thisYear,thisMonth,thisDay)
        if TodayDate<Promo2SinceDate:
            return 0
        else:
            return (TodayDate-Promo2SinceDate).days
data_train_store['DaysSincePromo2']=data_train_store.apply(
    lambda row:calc_DaysSincePromo2(row['ThisYear'],row['ThisMonth'],row['ThisDay'],row['Promo2'],row['Promo2SinceYear'],row['Promo2SinceWeek']),axis=1) 
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
def calc_IsPromo2Interval(thisMonth,Promo2,DaysSincePromo2,PromoInterval):
    if Promo2==0:
        return 0
    else:
        if not DaysSincePromo2>0:
            return 0
        else:
            if (PromoInterval=='Jan,Apr,Jul,Oct' )and (thisMonth==1 or thisMonth==4 or thisMonth==7 or thisMonth==10):
                return 1
            elif (PromoInterval=='Feb,May,Aug,Nov') and (thisMonth==2 or thisMonth==5 or thisMonth==8 or thisMonth==11):
                return 1
            elif (PromoInterval=='Mar,Jun,Sept,Dec') and (thisMonth==3 or thisMonth==6 or thisMonth==9 or thisMonth==12):
                return 1
            else:
                return 0
data_train_store['IsPromo2Interval']=data_train_store.apply(
    lambda row:calc_IsPromo2Interval(row['ThisMonth'],row['Promo2'],row['DaysSincePromo2'],row['PromoInterval']),axis=1) 
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Calc DaysInPromoCycle
start_time = time.time()
data_train_store['DaysInPromoCycle']=0
store_id=1
count_id=0
for index,row in (data_train_store.sort_values(by=['Store','Date'],ascending=[True,True])).iterrows():
#     print(index)
    # if store_id change, reset count_id=0, and change store_id to the new one
    if not int(row['Store'])==int(store_id):
        count_id=0
        store_id=row['Store']
    # if not open, DaysInPromoCycle=0

#     pdb.set_trace()
    if row['Open']==0:
        data_train_store.set_value(index,'DaysInPromoCycle',0)
#         print('store_id='+str(store_id)+'\topen='+str(row['Open'])+'\tPromo='+str(row['Promo'])+'\tcount_id='+str(count_id)+'\tDaysInPromoCycle='+ str(data_train_store.iloc[index]['DaysInPromoCycle']))            
        
    else:
        # if not Promo, DaysInPromoCycle=0        
        if row['Promo']==0:
            count_id=0
            data_train_store.set_value(index,'DaysInPromoCycle',0)
#             print('store_id='+str(store_id)+'\topen='+str(row['Open'])+'\tPromo='+str(row['Promo'])+'\tcount_id='+str(count_id)+'\tDaysInPromoCycle='+ str(data_train_store.iloc[index]['DaysInPromoCycle']))            
        # if Promo, DaysInPromoCycle=current_count_id+1        
        else:
            count_id+=1
            data_train_store.set_value(index,'DaysInPromoCycle',count_id)
#             print('store_id='+str(store_id)+'\topen='+str(row['Open'])+'\tPromo='+str(row['Promo'])+'\tcount_id='+str(count_id)+'\tDaysInPromoCycle='+ str(data_train_store.iloc[index]['DaysInPromoCycle']))            
# display(data_train_store.sort_values(by=['Store','Date'],ascending=[True,True])[['Store','Date','Open','Promo','DaysInPromoCycle']])
#         print 'index='+str(index)+'\tstore_id='+str(row['Store'])+'\tDate='+str(row['Date'])+'\tOpen='+str(row['Open'])+'\tPromo='+str(row['Promo'])+'\tDays='+str(row['DaysInPromoCycle'])
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#Calc DaysPostPromoCycle
start_time = time.time()
data_train_store['DaysPostPromoCycle']=0
store_id=1
count_id=0
for index,row in (data_train_store.sort_values(by=['Store','Date'],ascending=[True,True])).iterrows():
    if not int(row['Store'])==int(store_id):
        count_id=0
        store_id=row['Store']

    if row['Open']==0:
        data_train_store.set_value(index,'DaysPostPromoCycle',0)        
    else:
        if row['Promo']==1:
            count_id=0
            data_train_store.set_value(index,'DaysPostPromoCycle',0)
        else:
            count_id+=1
            data_train_store.set_value(index,'DaysPostPromoCycle',count_id)
#data_train_store[900:950][['Store','Date','Open','Promo','DaysInPromoCycle','DaysPostPromoCycle']]
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#create boolen value of  StateHoliday ='a','b','c'
start_time = time.time()
data_train_store['StateHoliday_a']=data_train_store['StateHoliday'].apply(lambda x:x=='a')
data_train_store['StateHoliday_b']=data_train_store['StateHoliday'].apply(lambda x:x=='b')
data_train_store['StateHoliday_c']=data_train_store['StateHoliday'].apply(lambda x:x=='c')
data_train_store['IsStateHoliday']=data_train_store['StateHoliday'].apply(lambda x:x=='c' or x=='b' or x=='a')
data_train_store['IsHoliday_Today']=data_train_store['IsStateHoliday'] | data_train_store['SchoolHoliday']
data_IsHolidayToday=(data_train_store[['Store','Today_ISOYear','Today_ISOWeek','IsHoliday_Today']].groupby(['Store','Today_ISOYear','Today_ISOWeek']).any())
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()

data_IsHolidayThisWeek=data_IsHolidayToday.reset_index(level=['Store','Today_ISOYear','Today_ISOWeek'])
#ThisWeek 
data_IsHolidayThisWeek.rename(columns={'IsHoliday_Today':'IsHolidayThisWeek'},inplace=True)
data_train_store=data_train_store.merge(data_IsHolidayThisWeek,left_on=['Store','Today_ISOYear','Today_ISOWeek'],right_on=['Store','Today_ISOYear','Today_ISOWeek'],how='left')
#NextWeek
data_IsHolidayThisWeek.rename(columns={'IsHolidayThisWeek':'IsHolidayNextWeek','Today_ISOYear':'NextWeek_ISOYear','Today_ISOWeek':'NextWeek_ISOWeek'},inplace=True)
data_train_store=data_train_store.merge(data_IsHolidayThisWeek,left_on=['Store','NextWeek_ISOYear','NextWeek_ISOWeek'],right_on=['Store','NextWeek_ISOYear','NextWeek_ISOWeek'],how='left')
#LastWeek
data_IsHolidayThisWeek.rename(columns={'IsHolidayNextWeek':'IsHolidayLastWeek','NextWeek_ISOYear':'LastWeek_ISOYear','NextWeek_ISOWeek':'LastWeek_ISOWeek'},inplace=True)
data_train_store=data_train_store.merge(data_IsHolidayThisWeek,left_on=['Store','LastWeek_ISOYear','LastWeek_ISOWeek'],right_on=['Store','LastWeek_ISOYear','LastWeek_ISOWeek'],how='left')

# The decision tree will handle the missing value automatically
# data_train_store['IsHolidayNextWeek'].fillna(False,inplace=True)
# data_train_store['IsHolidayLastWeek'].fillna(False,inplace=True)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# create new sales and new customers which bfill the 0 
start_time = time.time()
data_train_store[['Sales_New']]=(data_train_store[['Store','Sales']].replace(0,np.nan)).groupby(['Store']).fillna(method='bfill')
grouped=data_train_store[['Store','Sales_New']].sort_index(ascending=False).groupby(['Store'])
def calc_MovingMedian_NDays(input_grouped,NDays=3):
    assert (NDays>=1)
    temp=input_grouped.apply(pd.rolling_median,NDays)
    temp.index=temp.index.droplevel(0)
    return temp
def calc_MovingAverage_NDays(input_grouped,NDays=3):
    assert (NDays>=1)
    temp=input_grouped.apply(pd.rolling_mean,NDays)
    temp.index=temp.index.droplevel(0)
    return temp
def calc_MovingStd_NDays(input_grouped,NDays=3):
    assert (NDays>=1)
    temp=input_grouped.apply(pd.rolling_std,NDays)
    temp.index=temp.index.droplevel(0)
    return temp


data_train_store[['Sales_MM_3Days']]=calc_MovingMedian_NDays(grouped,NDays=3)[['Sales_New']]
data_train_store[['Sales_MM_7Days']]=calc_MovingMedian_NDays(grouped,NDays=7)[['Sales_New']]
data_train_store[['Sales_MM_14Days']]=calc_MovingMedian_NDays(grouped,NDays=14)[['Sales_New']]
data_train_store[['Sales_MM_30Days']]=calc_MovingMedian_NDays(grouped,NDays=30)[['Sales_New']]
data_train_store[['Sales_MM_90Days']]=calc_MovingMedian_NDays(grouped,NDays=90)[['Sales_New']]
data_train_store[['Sales_MM_180Days']]=calc_MovingMedian_NDays(grouped,NDays=180)[['Sales_New']]
data_train_store[['Sales_MM_365Days']]=calc_MovingMedian_NDays(grouped,NDays=365)[['Sales_New']]

data_train_store[['Sales_MA_3Days']]=calc_MovingAverage_NDays(grouped,NDays=3)[['Sales_New']]
data_train_store[['Sales_MA_7Days']]=calc_MovingAverage_NDays(grouped,NDays=7)[['Sales_New']]
data_train_store[['Sales_MA_14Days']]=calc_MovingAverage_NDays(grouped,NDays=14)[['Sales_New']]    
data_train_store[['Sales_MA_30Days']]=calc_MovingAverage_NDays(grouped,NDays=30)[['Sales_New']]
data_train_store[['Sales_MA_90Days']]=calc_MovingAverage_NDays(grouped,NDays=90)[['Sales_New']]
data_train_store[['Sales_MA_180Days']]=calc_MovingAverage_NDays(grouped,NDays=180)[['Sales_New']]
data_train_store[['Sales_MA_365Days']]=calc_MovingAverage_NDays(grouped,NDays=365)[['Sales_New']]

data_train_store[['Sales_MStd_3Days']]=calc_MovingStd_NDays(grouped,NDays=3)[['Sales_New']]
data_train_store[['Sales_MStd_7Days']]=calc_MovingStd_NDays(grouped,NDays=7)[['Sales_New']]
data_train_store[['Sales_MStd_14Days']]=calc_MovingStd_NDays(grouped,NDays=14)[['Sales_New']]    
data_train_store[['Sales_MStd_30Days']]=calc_MovingStd_NDays(grouped,NDays=30)[['Sales_New']]
data_train_store[['Sales_MStd_90Days']]=calc_MovingStd_NDays(grouped,NDays=90)[['Sales_New']]
data_train_store[['Sales_MStd_180Days']]=calc_MovingStd_NDays(grouped,NDays=180)[['Sales_New']]
data_train_store[['Sales_MStd_365Days']]=calc_MovingStd_NDays(grouped,NDays=365)[['Sales_New']]


# ((data_train_store[['Store','Sales','Customers']].replace(0,np.nan)).groupby(['Store'])).fillna(method='bfill',inplace=True)
# data_train_store[['Store','Sales_New','Customers_New']].groupby(['Store']).fillna(method='bfill',inplace=True)
print("--- %s seconds ---" % (time.time() - start_time))
# data_train_store.head(n=10)


In [219]:
data_train_store

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,LogCompetitionDistance,shopsales_mo_avg,shopsales_tu_avg,shopsales_we_avg,shopsales_th_avg,shopsales_fr_avg,shopsales_sa_avg,shopsales_mo_med,shopsales_tu_med,shopsales_we_med,shopsales_th_med,shopsales_fr_med,shopsales_sa_med,shopsales_mo_std,shopsales_tu_std,shopsales_we_std,shopsales_th_std,shopsales_fr_std,shopsales_sa_std,shopsales_mo_25%,shopsales_tu_25%,shopsales_we_25%,shopsales_th_25%,shopsales_fr_25%,shopsales_sa_25%,shopsales_mo_75%,shopsales_tu_75%,shopsales_we_75%,shopsales_th_75%,shopsales_fr_75%,shopsales_sa_75%,shop_customers_mean,shop_customers_median,shop_customers_std,shop_customers_25%,shop_customers_75%,shop_salespercustomer,shop_salespercustomer_mean,shop_salespercustomer_median,shop_salespercustomer_std,shop_salespercustomer_25%,shop_salespercustomer_75%,shop_schoolholiday_mean,shop_schoolholiday_median,shop_schoolholiday_std,shop_schoolholiday_25%,shop_schoolholiday_75%,shopsales_promo_mean,shopsales_promo_median,shopsales_promo_std,shopsales_promo_25%,shopsales_promo_75%,ThisDay,ThisMonth,ThisYear,Today_ISOWeek,Today_ISOYear,DaysOfYear,Day_LastWeek,Month_LastWeek,Year_LastWeek,LastWeek_ISOWeek,LastWeek_ISOYear,Day_NextWeek,Month_NextWeek,Year_NextWeek,NextWeek_ISOWeek,NextWeek_ISOYear,OpenDaysSinceCompetition,DaysSincePromo2,IsPromo2Interval,DaysInPromoCycle,DaysPostPromoCycle,StateHoliday_a,StateHoliday_b,StateHoliday_c,IsStateHoliday,IsHoliday_Today,IsHolidayThisWeek,IsHolidayNextWeek,IsHolidayLastWeek,Date_50DaysAgo,Date_92DaysAgo,Date_182DaysAgo,Date_365DaysAgo,Sales_New,Customers_New,Sales_3Days,Customers_3Days,Sales_MM_3Days,Customers_MM_3Days,Sales_MM_7Days,Sales_MM_14Days,Sales_MM_30Days,Sales_MM_90Days,Sales_MM_180Days,Sales_MM_360Days,Sales_MM_365Days,Sales_MA_3Days,Sales_MA_7Days,Sales_MA_14Days,Sales_MA_30Days,Sales_MA_90Days,Sales_MA_180Days,Sales_MA_365Days,Sales_MStd_3Days,Sales_MStd_7Days,Sales_MStd_14Days,Sales_MStd_30Days,Sales_MStd_90Days,Sales_MStd_180Days,Sales_MStd_365Days
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9,2008,0,NaN,NaN,NaN,0.636024,4977.50,4315.25,4202.50,4278.25,4407.25,4100.000000,4724.5,4330.5,4282.0,4162.0,4330.0,4364.0,814.903062,822.236533,674.362168,570.674090,770.489617,494.080965,4386.00,3627.00,3713.75,3865.00,3782.50,3947.0,5316.00,5018.75,4770.75,4575.25,4954.75,4385.0,512.652174,517.0,42.318714,482.5,545.0,9.702400,9.194139,9.194139,NaN,9.194139,9.194139,5235.600000,5020.0,513.342283,5011.00,5263.0,5032.0,5015.5,438.504276,4799.50,5051.00,31,7,2015,31,2015,211,24,7,2015,30,2015,7,8,2015,32,2015,2524,0,0,5,0,False,False,False,False,True,True,False,False,2015-06-11,2015-04-30,2015-01-30,2014-07-31,5263.0,5263.0,546.0,5020.0,5020.0,5020.0,5011.0,4400.5,NaN,NaN,NaN,NaN,NaN,5021.666667,4986.571429,4472.142857,NaN,NaN,NaN,NaN,240.504331,597.124738,730.642484,NaN,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9,2008,0,NaN,NaN,NaN,0.636024,4977.50,4315.25,4202.50,4278.25,4407.25,4100.000000,4724.5,4330.5,4282.0,4162.0,4330.0,4364.0,814.903062,822.236533,674.362168,570.674090,770.489617,494.080965,4386.00,3627.00,3713.75,3865.00,3782.50,3947.0,5316.00,5018.75,4770.75,4575.25,4954.75,4385.0,512.652174,517.0,42.318714,482.5,545.0,9.702400,9.194139,9.194139,NaN,9.194139,9.194139,5235.600000,5020.0,513.342283,5011.00,5263.0,5032.0,5015.5,438.504276,4799.50,5051.00,30,7,2015,31,2015,210,23,7,2015,30,2015,6,8,2015,32,2015,2523,0,0,4,0,False,False,False,False,True,True,False,False,2015-06-10,2015-04-29,2015-01-29,2014-07-30,5020.0,5020.0,546.0,5011.0,5011.0,5011.0,4782.0,4400.5,NaN,NaN,NaN,NaN,NaN,4937.666667,4764.142857,4442.785714,NaN,NaN,NaN,NaN,134.886372,747.939041,704.200057,NaN,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9,2008,0,NaN,NaN,NaN,0.636024,4977.50,4315.25,4202.50,4278.25,4407.25,4100.000000,4724.5,4330.5,4282.0,

In [143]:
#stop use so complex method
# # skip open=0 (OpenRate=0.83)
# # calc moving average
# OpenRate=0.83
# day_7=round(7.0*OpenRate)
# day_15=round(15.0*OpenRate)
# day_30=round(30.0*OpenRate)
# day_60=round(60.0*OpenRate)
# day_90=round(90.0*OpenRate)
# day_180=round(180.0*OpenRate)
# day_365=round(365.0*OpenRate)

In [ ]:
#

In [ ]:
data_train_store[['Store','Date','Date_50DaysAgo','Date_92DaysAgo','Date_182DaysAgo','Date_365DaysAgo']].head()[]

In [125]:
temp=data_train_store[0:1000]

In [83]:
temp_data.head()

,Store,Open,Date
0,1,1,2015-07-31
1,1,1,2015-07-30
2,1,1,2015-07-29
3,1,1,2015-07-28
4,1,1,2015-07-27


In [91]:
temp_data=data_train_store[['Store','Open','Date']]
selected_row=((temp_data['Store']==1)&(temp_data['Date']=='2015-07-26'))
display(temp_data.head(10))
print selected_row.any()
print temp_data[selected_row]
print temp_data[selected_row]['Date']

,Store,Open,Date
0,1,1,2015-07-31
1,1,1,2015-07-30
2,1,1,2015-07-29
3,1,1,2015-07-28
4,1,1,2015-07-27
5,1,0,2015-07-26
6,1,1,2015-07-25
7,1,1,2015-07-24
8,1,1,2015-07-23
9,1,1,2015-07-22


True
   Store  Open        Date
5      1     0  2015-07-26
5    2015-07-26
Name: Date, dtype: object


In [118]:
## calc the date of given date N days ago, on which day the store is open  stop-use because of slow speed
# start_time = time.time()
# temp_data=data_train_store[['Store','Open','Date']]
# def calc_NDaysAgo(df,input_date,store_id=1,n_Days=1,maxLoop=0):
#     assert int(n_Days)>int(maxLoop)
#     count_loop=0
#     newDate_=(datetime.datetime.strptime(str(input_date),"%Y-%m-%d")-datetime.timedelta(n_Days))
#     newDate=newDate_.strftime("%Y-%m-%d")    
#     while(count_loop<=maxLoop):
#         selected_row=((df['Date']==newDate) &( df['Store']==store_id) & df['Open']==1)
#         if selected_row.any():
#             return newDate
#         else:
#             count_loop+=1
#             newDate_=newDate_-datetime.timedelta(1)
#             newDate=newDate_.strftime("%Y-%m-%d")        
#     return np.nan


# # print calc_NDaysAgo(temp_data,'2015-07-31',store_id=1,n_Days=2,maxLoop=0)

# for index,row in temp_data.iterrows():
#     temp_data.set_value(index,'NewDate',calc_NDaysAgo(temp_data,row['Date'],row['Store'],n_Days=5,maxLoop=1))
# print("--- %s seconds ---" % (time.time() - start_time))

--- 99.8382451534 seconds ---


In [ ]:
                        
    #     calc_NDaysAgo(temp_data,input_date=row['Date'],n_Days=2,maxLoop=2)

#,store_id=row['Store'],n_Days=2,maxLoop=1)
    
# temp_data   

In [ ]:
#HolidayThisWeek
for index,row in data_train_store.iterrows():
    print index
    temp_key=(row['Store'],row['ThisYear'],row['ThisWeek'])
    data_train_store[index,'HolidayThisWeek']=holiday_thisweek_dict[temp_key]

list all possible features

| 1|1 |
|:-:|:-:|
|1|1|

- competition
    - CompetitionDistance  for store
    - LogCompetitionDistance for store
    - CompetitionOpenSinceMonth for store
    - CompetitionOpenSinceYear for store 
    - DaysSinceCompetition for store & day 
    - RndDaysSinceCompetition for store & day
- promo2
    - Promo2	
    - Promo2SinceWeek(NaN)
    - Promo2SinceYear	
    - PromoInterval
- promo
    - promo
    -'DaysInPromoCycle'
    - 'DaysPostPromoCycle'
    - 'DaysPrePromoCycle'
    - 
- holiday    

- trend - machine learning indicators
    - momentum(50 days,92 days,183days,365 days)
    - moving average(one day,three days, one week,two week, one month,one quarter, half a year, one year) 加权移动平均 
    - std deviation(, one week)
    


recent data: 

temporal information
current trends
other info

holiday

promo







current month
current week
year
past_past week/month/quarter/year


In [244]:
tt=data_train[(data_train['Open']==1)].sort(['Store','Date'],ascending=[True,True])
#Store change
# df['dA'] = df['A'] - df['A'].shift(-1)

tt['Delta_Store']=((tt['Store']-tt['Store'].shift(1)).fillna(0)!=0)*1.0
tt


/Users/pengjuzhao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,dayofyear,month,day,year,week,Delta_Store
1014980,1,3,2013-01-02,5530,668,1,0,0,1,1,1,2,2013,1,0.0
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2,1,3,2013,1,0.0
1012750,1,5,2013-01-04,4486,619,1,0,0,1,3,1,4,2013,1,0.0
1011635,1,6,2013-01-05,4997,635,1,0,0,1,4,1,5,2013,1,0.0
1009405,1,1,2013-01-07,7176,785,1,1,0,1,6,1,7,2013,2,0.0
1008290,1,2,2013-01-08,5580,654,1,1,0,1,7,1,8,2013,2,0.0
1007175,1,3,2013-01-09,5471,626,1,1,0,1,8,1,9,2013,2,0.0
1006060,1,4,2013-01-10,4892,615,1,1,0,1,9,1,10,2013,2,0.0
1004945,1,5,2013-01-11,4881,592,1,1,0,1,10,1,11,2013,2,0.0
1003830,1,6,2013-01-12,4952,646,1,0,0,0,11,1,12,2013,2,0.0


In [230]:
np.logical_and(1,True)+1

2

In [223]:
#daysinpromocycle


#prempromocycle

tt=data_train[(data_train['Store']==1)& (data_train['Open']==1)]
tt

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,dayofyear,month,day,year,week
0,1,5,2015-07-31,5263,555,1,1,0,1,211,7,31,2015,31
1115,1,4,2015-07-30,5020,546,1,1,0,1,210,7,30,2015,31
2230,1,3,2015-07-29,4782,523,1,1,0,1,209,7,29,2015,31
3345,1,2,2015-07-28,5011,560,1,1,0,1,208,7,28,2015,31
4460,1,1,2015-07-27,6102,612,1,1,0,1,207,7,27,2015,31
6690,1,6,2015-07-25,4364,500,1,0,0,0,205,7,25,2015,30
7805,1,5,2015-07-24,3706,459,1,0,0,0,204,7,24,2015,30
8920,1,4,2015-07-23,3769,503,1,0,0,0,203,7,23,2015,30
10035,1,3,2015-07-22,3464,463,1,0,0,0,202,7,22,2015,30
11150,1,2,2015-07-21,3558,469,1,0,0,0,201,7,21,2015,30


In [225]:
tt.sort('Date',ascending=True)


/Users/pengjuzhao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,dayofyear,month,day,year,week
1014980,1,3,2013-01-02,5530,668,1,0,0,1,1,1,2,2013,1
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2,1,3,2013,1
1012750,1,5,2013-01-04,4486,619,1,0,0,1,3,1,4,2013,1
1011635,1,6,2013-01-05,4997,635,1,0,0,1,4,1,5,2013,1
1009405,1,1,2013-01-07,7176,785,1,1,0,1,6,1,7,2013,2
1008290,1,2,2013-01-08,5580,654,1,1,0,1,7,1,8,2013,2
1007175,1,3,2013-01-09,5471,626,1,1,0,1,8,1,9,2013,2
1006060,1,4,2013-01-10,4892,615,1,1,0,1,9,1,10,2013,2
1004945,1,5,2013-01-11,4881,592,1,1,0,1,10,1,11,2013,2
1003830,1,6,2013-01-12,4952,646,1,0,0,0,11,1,12,2013,2


In [226]:
tt.sort('Date',ascending=False)

/Users/pengjuzhao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,dayofyear,month,day,year,week
0,1,5,2015-07-31,5263,555,1,1,0,1,211,7,31,2015,31
1115,1,4,2015-07-30,5020,546,1,1,0,1,210,7,30,2015,31
2230,1,3,2015-07-29,4782,523,1,1,0,1,209,7,29,2015,31
3345,1,2,2015-07-28,5011,560,1,1,0,1,208,7,28,2015,31
4460,1,1,2015-07-27,6102,612,1,1,0,1,207,7,27,2015,31
6690,1,6,2015-07-25,4364,500,1,0,0,0,205,7,25,2015,30
7805,1,5,2015-07-24,3706,459,1,0,0,0,204,7,24,2015,30
8920,1,4,2015-07-23,3769,503,1,0,0,0,203,7,23,2015,30
10035,1,3,2015-07-22,3464,463,1,0,0,0,202,7,22,2015,30
11150,1,2,2015-07-21,3558,469,1,0,0,0,201,7,21,2015,30


In [170]:
data_train.head(n=5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,dayofyear,month,day,year
0,1,5,2015-07-31,5263,555,1,1,0,1,31,7,31,2015
1,2,5,2015-07-31,6064,625,1,1,0,1,31,7,31,2015
2,3,5,2015-07-31,8314,821,1,1,0,1,31,7,31,2015
3,4,5,2015-07-31,13995,1498,1,1,0,1,31,7,31,2015
4,5,5,2015-07-31,4822,559,1,1,0,1,31,7,31,2015


In [162]:
data_train['dayofyear']=data_train['DateTime'].apply()
data_train.head(n=5)

TypeError: 'getset_descriptor' object is not callable

In [ ]:
datetime.datetime.day()

In [153]:
print(str(raw_data_train.Date))

0          2015-07-31
1          2015-07-31
2          2015-07-31
3          2015-07-31
4          2015-07-31
5          2015-07-31
6          2015-07-31
7          2015-07-31
8          2015-07-31
9          2015-07-31
10         2015-07-31
11         2015-07-31
12         2015-07-31
13         2015-07-31
14         2015-07-31
15         2015-07-31
16         2015-07-31
17         2015-07-31
18         2015-07-31
19         2015-07-31
20         2015-07-31
21         2015-07-31
22         2015-07-31
23         2015-07-31
24         2015-07-31
25         2015-07-31
26         2015-07-31
27         2015-07-31
28         2015-07-31
29         2015-07-31
              ...    
1017179    2013-01-01
1017180    2013-01-01
1017181    2013-01-01
1017182    2013-01-01
1017183    2013-01-01
1017184    2013-01-01
1017185    2013-01-01
1017186    2013-01-01
1017187    2013-01-01
1017188    2013-01-01
1017189    2013-01-01
1017190    2013-01-01
1017191    2013-01-01
1017192    2013-01-01
1017193   

In [151]:
bb=[datetime.datetime.strptime(x, "%Y-%m-%d") for x in str(raw_data_train.Date)]

ValueError: time data '0' does not match format '%Y-%m-%d'

In [78]:
pd.Series((raw_data_train[(raw_data_train['Open']==1)]['Sales']))

0           5263
1           6064
2           8314
3          13995
4           4822
5           5651
6          15344
7           8492
8           8565
9           7185
10         10457
11          8959
12          8821
13          6544
14          9191
15         10231
16          8430
17         10071
18          8234
19          9593
20          9515
21          6566
22          7273
23         14190
24         14180
25          7818
26         13213
27          7301
28          9411
29          4776
           ...  
1016082     3154
1016083     4161
1016084     2814
1016085     5099
1016086     3955
1016087     6220
1016088     4576
1016089     4126
1016090     5097
1016091    10797
1016092     6218
1016093    20642
1016094     3697
1016179     4220
1016353     6851
1016356    17267
1016368     3102
1016429     2401
1016447     3139
1016517     9643
1016588     3113
1016606     2646
1016624     2907
1016656     8498
1016770     3821
1016776     3375
1016827    10765
1016863     50

In [126]:
# data_train['dayofyear']=raw_data_train['Date']
data_train.head(n=10)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
5,6,5,2015-07-31,5651,589,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
7,8,5,2015-07-31,8492,833,1,1,0,1
8,9,5,2015-07-31,8565,687,1,1,0,1
9,10,5,2015-07-31,7185,681,1,1,0,1


In [129]:
datetime.date(data_train['Date'])

TypeError: cannot convert the series to <type 'int'>

In [26]:
#find rows with null value
data_test[data_test.isnull().any(axis=1)]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


In [27]:
data_train[data_train.isnull().any(axis=1)]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday


In [28]:
len(data_test)

41088

{'CompetitionDistance': [1, 1, 1, 0],
 'CompetitionOpenSinceMonth': [0, 0, 1, 0],
 'CompetitionOpenSinceYear': [1, 1, 1, 0],
 'Promo2': [0, 0, 1, 0],
 'Promo2SinceWeek': [0, 0, 1, 0],
 'Promo2SinceYear': [1, 1, 1, 0],
 'assortment': [1, 1, 1, 0],
 'closurefeat': [1, 1, 1, 0],
 'day': [1, 1, 1, 0],
 'dayavg_openyesterday': [0, 0, 1, 0],
 'dayofweek': [1, 1, 1, 0],
 'dayofyear': [1, 1, 1, 0],
 'daysincompetition': [0, 0, 1, 0],
 'daysincompetition_unrounded': [1, 1, 1, 0],
 'daysinpromocycle': [1, 1, 1, 0],
 'holidays_lastweek': [1, 1, 1, 0],
 'holidays_nextweek': [1, 1, 1, 0],
 'holidays_thisweek': [1, 1, 1, 0],
 'lastmonth_yoy': [0, 0, 0, 0],
 'linmod_quarterly': [0, 0, 0, 1],
 'linmod_yearly': [0, 0, 0, 1],
 'month': [0, 0, 1, 0],
 'prevhalfyear': [0, 0, 0, 1],
 'prevhalfyear_m1': [0, 0, 0, 1],
 'prevhalfyear_m3': [0, 0, 0, 0],
 'prevquarter_cust_dps_med': [0, 1, 0, 1],
 'prevquarter_dps_med': [0, 0, 0, 1],
 'prevquarter_ds_hmean': [0, 0, 0, 0],
 'prevquarter_hmean': [0, 0, 0, 0],
 'p

In [39]:
len(features.keys())

62

NameError: name 'a' is not defined